# MobileNetV2

### Part 1 - Building the CNN

#### Importing the Tensorflow libraries and packages

In [ ]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau

import os

##  MobileNetV2

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.applications import MobileNetV2

conv_model = MobileNetV2(include_top=False, weights='imagenet',input_shape=(150,150,3))

conv_model.summary()

In [ ]:
import matplotlib.pyplot as plt
def plot_history(history, yrange):
    '''Plot loss and accuracy as a function of the epoch,
    for the training and validation datasets.
    '''
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Get number of epochs
    epochs = range(len(acc))

    # Plot training and validation accuracy per epoch
    plt.plot(epochs, acc)
    plt.plot(epochs, val_acc)
    plt.title('Training and validation accuracy')
    plt.ylim(yrange)
    
    # Plot training and validation loss per epoch
    plt.figure()

    plt.plot(epochs, loss)
    plt.plot(epochs, val_loss)
    plt.title('Training and validation loss')
    
    plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32

def generators(shape): 
    '''Create the training and validation datasets for 
    a given image shape.
    '''
    path_dir = '../../dataset/train'
    parth_dir_test = '../../dataset/test'

    datagen = ImageDataGenerator(
                    rescale=1./255,
                    #preprocessing_function=tensorflow.keras.applications.mobilenet_v2.preprocess_input,
                    validation_split = 0.2)

    train_dataset = datagen.flow_from_directory(
        path_dir,
        target_size=(150,150),
        shuffle=True,
        subset='training',
        batch_size = 64,
        seed = 11,
        class_mode = 'binary'
    )
    
    val_dataset = datagen.flow_from_directory(
        path_dir,
        target_size=(150,150),
        subset='validation',
        batch_size = 64,
        seed = 11,
        class_mode = 'binary'
    )

    test_dataset = datagen.flow_from_directory(parth_dir_test,
                                        target_size= (150,150),
                                        batch_size = 64,
                                        class_mode = 'binary',
                                        seed = 11,
                                        shuffle=False)
    return train_dataset, val_dataset, test_dataset

In [ ]:
train_dataset, val_dataset, test_dataset = generators((150,150))

In [ ]:
train_dataset.class_indices

In [ ]:
# flatten the output of the convolutional part: 
x = keras.layers.Flatten()(conv_model.output)
# three hidden layers
x = keras.layers.Dense(16, activation='relu')(x)
x = keras.layers.Dropout(0.20)(x)
x = keras.layers.Dense(32, activation='relu')(x)
x = keras.layers.Dropout(0.25)(x)

# final softmax layer
predictions = keras.layers.Dense(1, activation='sigmoid')(x)

# creating the full model:
full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
full_model.summary()

In [ ]:
for layer in conv_model.layers:
    layer.trainable = False

In [ ]:
full_model.summary()

In [ ]:
full_model.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.Adamax(lr=0.001),
                  metrics=['acc'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import datetime

#rm -rf /logs/

es = EarlyStopping(monitor='val_loss', patience=4)
mc = ModelCheckpoint('best_mobilenetv2.h5', monitor='val_loss', mode='min', save_best_only=True)
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

# Create a TensorBoard callback
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = full_model.fit_generator(
    train_dataset, 
    validation_data = val_dataset,
    callbacks = [tensorboard_callback,es,mc,reduceLR],
    epochs=25
)

In [ ]:
plot_history(history, yrange=(0,1))

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
full_model.load_weights('best_mobilenetv2.h5')

In [ ]:
full_model.summary()

In [ ]:
score = full_model.evaluate(test_dataset, verbose=0)

print('Test Loss', score[0])
print('Test accuracy', score[1])

In [ ]:
#Overfitting -> Your training accuracy will be higher than the accuracy on the validation/test set

#Overfitting indicates that your model is too complex for the problem that it is solving,
#filters in the case of Convolutional Neural Networks, and layers in the case of overall Deep Learning Models

#How do you know if your model is underfitting? Your model is underfitting if the accuracy on the validation set 
#is higher than the accuracy on the training set. Additionally, 
#if the whole model performs bad this is also called underfitting.

#https://towardsdatascience.com/deep-learning-3-more-on-cnns-handling-overfitting-2bd5d99abe5d

In [ ]:
test_dataset.classes

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

yhat_classes = full_model.predict(test_dataset, verbose=0)
yhat_classes = yhat_classes[:, 0]

print(classification_report(test_dataset.classes,yhat_classes.round()))